In [1]:
import urllib.request
from pandas import Series, DataFrame
from pandas.io.html import read_html
import pyparsing as py
from pyparsing import (Word, Literal, Group, alphanums, OneOrMore, nums, Combine, Optional,
                       FollowedBy, StringEnd, alphas, SkipTo)
import re

# Departamentos and paginas web
Esta lista la edite manualmente de la web: 
http://plebiscito.registraduria.gov.co/99PL/DPLZZZZZZZZZZZZZZZZZ_L1.htm

In [2]:
depts =\
{"AMAZONAS": "DPL60ZZZZZZZZZZZZZZZ_L1.htm",  
"ANTIOQUIA": "DPL01ZZZZZZZZZZZZZZZ_L1.htm",  
"ARAUCA": "DPL40ZZZZZZZZZZZZZZZ_L1.htm",  
"ATLANTICO": "DPL03ZZZZZZZZZZZZZZZ_L1.htm",   
"BOLIVAR": "DPL05ZZZZZZZZZZZZZZZ_L1.htm",  
"BOYACA": "DPL07ZZZZZZZZZZZZZZZ_L1.htm",  
"CALDAS": "DPL09ZZZZZZZZZZZZZZZ_L1.htm",  
"CAQUETA": "DPL44ZZZZZZZZZZZZZZZ_L1.htm",  
"CASANARE": "DPL46ZZZZZZZZZZZZZZZ_L1.htm",  
"CAUCA": "DPL11ZZZZZZZZZZZZZZZ_L1.htm",  
"CESAR": "DPL12ZZZZZZZZZZZZZZZ_L1.htm",  
"CHOCO": "DPL17ZZZZZZZZZZZZZZZ_L1.htm",  
"CONSULADOS": "DPL88ZZZZZZZZZZZZZZZ_L1.htm",  
"CORDOBA": "DPL13ZZZZZZZZZZZZZZZ_L1.htm",  
"CUNDINAMARCA": "DPL15ZZZZZZZZZZZZZZZ_L1.htm",  
"GUAINIA": "DPL50ZZZZZZZZZZZZZZZ_L1.htm",  
"GUAVIARE": "DPL54ZZZZZZZZZZZZZZZ_L1.htm",  
"HUILA": "DPL19ZZZZZZZZZZZZZZZ_L1.htm",  
"LA GUAJIRA": "DPL48ZZZZZZZZZZZZZZZ_L1.htm",  
"MAGDALENA": "DPL21ZZZZZZZZZZZZZZZ_L1.htm",  
"META": "DPL52ZZZZZZZZZZZZZZZ_L1.htm",  
"NARIÑO": "DPL23ZZZZZZZZZZZZZZZ_L1.htm",  
"NORTE DE SANTANDER": "DPL25ZZZZZZZZZZZZZZZ_L1.htm",  
"PUTUMAYO": "DPL64ZZZZZZZZZZZZZZZ_L1.htm",  
"QUINDIO": "DPL26ZZZZZZZZZZZZZZZ_L1.htm",  
"RISARALDA": "DPL24ZZZZZZZZZZZZZZZ_L1.htm",  
"SAN ANDRES": "DPL56ZZZZZZZZZZZZZZZ_L1.htm",  
"SANTANDER": "DPL27ZZZZZZZZZZZZZZZ_L1.htm",  
"SUCRE": "DPL28ZZZZZZZZZZZZZZZ_L1.htm",  
"TOLIMA": "DPL29ZZZZZZZZZZZZZZZ_L1.htm",  
"VALLE": "DPL31ZZZZZZZZZZZZZZZ_L1.htm",  
"VAUPES": "DPL68ZZZZZZZZZZZZZZZ_L1.htm",  
"VICHADA": "DPL72ZZZZZZZZZZZZZZZ_L1.htm",
"BOGOTA D.C.": "DPL16ZZZZZZZZZZZZZZZ_L1.htm"}

In [3]:
depts_df = DataFrame.from_dict(depts, orient='index')
depts_df['nombre'] = depts_df.index
depts_df.index = [x for x in range(len(depts_df.index))]
depts_df.columns = ['url', 'nombre']
depts_df['url'] = "http://plebiscito.registraduria.gov.co/99PL/" + depts_df['url'] 
depts_df = depts_df[['nombre', 'url']]

# Obtener todos los municipios

In [4]:
def get_municipios(dept):
    depto_url = depto['url']
    depto_html = urllib.request.urlopen(depto_url).read()
    municipio = re.compile('<option value="../\d+\w+/DPL(?P<name>\d+)ZZZZZZZZZZZZ_L1.htm">(\w+)</option>')
    municipios = municipio.findall(string=depto_html.decode())
    municipios = DataFrame(municipios)
    municipios.columns = ['Id', 'nombre']
    municipios['departamento'] = depto['nombre']
    municipios['url'] = "http://plebiscito.registraduria.gov.co/99PL/DPL" + municipios['Id'] + "ZZZZZZZZZZZZ_L1.htm"
    return municipios

In [5]:
municipios = DataFrame()
for i in depts_df.index:
    depto = depts_df.iloc[i]
    if not depto['nombre'] == 'BOGOTA D.C.':
        municipios = municipios.append(get_municipios(depto))
    else:
        depto['departamento'] = "BOGOTA D.C."
        depto['Id'] = "16ZZZ"
        municipios = municipios.append(depto)
# new index
municipios.index = range(len(municipios))

##  Parser para los datos por municipio

In [6]:
censo_txt = '8.455 de 31.054 personas habilitadas'
votos_txt =  '(5.234 Votos)'
validos_txt = '<b>Votos v\xc3\xa1lidos</b><br/>6.558'
nomarcados_txt = 'Votos no marcados</b><br/>163'
nulos_txt = 'Votos nulos</b><br/>286'

In [7]:
# Obtiene las personas habilitadas
integer_point = Combine(Word(nums) + Optional(OneOrMore(Literal('.').suppress() + Word(nums))))
ph_txt = Literal('personas habilitadas').suppress()
censo = integer_point('votos') + Literal('de').suppress() + integer_point('habilitados') + ph_txt

# Votos por el no y el si (primero si despues no )
rb = Literal(')').suppress()
lb = Literal('(').suppress()
voto = lb + integer_point('votos')+ Literal('Votos').suppress() +  rb

# Votos validos (mas facil)
validos = Literal('Votos válidos</b><br/>').suppress() + integer_point('validos')
# Votos no marcados
nomarcados = Literal('Votos no marcados</b><br/>').suppress() + integer_point('nomarcados')
# Votos nulos
nulos = Literal('Votos nulos</b><br/>').suppress() + integer_point('nulos')

In [8]:
def get_html(url):
    html = urllib.request.urlopen(url).read()
    return html

# def get_from_html(html, rule):
#     content = rule.searchString(html.decode('utf-8')).asList()
#     return content

def get_votos_habilidatos(html):
    content= censo.searchString(html.decode('utf-8')).asList()[0]
    return content

def get_votos(html):
    si, no = voto.searchString(html.decode('utf-8')).asList()
    return si[0], no[0]

def get_validos(html):
    val = validos.searchString(html.decode('utf-8')).asList()[0][0]
    return val

def get_nulos(html):
    nul = nulos.searchString(html.decode('utf-8')).asList()[0][0]
    return nul
    
def get_nomarcados(html):
    nomarcados_ = nomarcados.searchString(html.decode('utf-8')).asList()[0][0]
    return nomarcados_


def get_votos_municipios():
    print('get the html')
    municipios['html'] = municipios['url'].apply(get_html)
    print('get votos')
    municipios['votos'] = municipios['html'].apply(get_votos)
    municipios['Si'] = municipios['votos'].apply(lambda x: x[0])
    municipios['No'] = municipios['votos'].apply(lambda x: x[1])
    print('get validos')
    municipios['validos'] = municipios['html'].apply(get_validos)
    print('get nulos')
    municipios['nulos'] = municipios['html'].apply(get_nulos)
    print('get nomarcados')
    municipios['nomarcados'] = municipios['html'].apply(get_nomarcados)
    print('get total votos/ habilitados')
    municipios['total votos/habilitados'] = municipios['html'].apply(get_votos_habilidatos)
    municipios['total votos'] = municipios['total votos/habilitados'].apply(lambda x: x[0])
    municipios['habilitados'] = municipios['total votos/habilitados'].apply(lambda x: x[1])

In [9]:
get_votos_municipios()

get the html
get votos
get validos
get nulos
get nomarcados
get total votos/ habilitados


Preserve

In [35]:
import time
day = time.strftime("%Y%m%d")
cols_sel = municipios.columns.difference(['html', 'total votos/habilitados', 'votos', 'url'])

In [36]:
municipios[cols_sel].to_csv('plebicito_municipios_{}.csv'.format(day))
municipios[cols_sel].to_excel('plebicito_municipios_{}.xls'.format(day))
municipios.to_pickle('plebicito_municipios_{}.pickle'.format(day))